This notebook seems to attempt to explain how the hyperparameter beta can be derived from the internal representation of the transformer.

In [ ]:
import matplotlib.pyplot as plt
import os 
import json
import sys
import numpy as np
sys.path.append("..")
import copy

In [ ]:
from lre import models, data, operators, functional, metrics, logging_utils
import logging
import torch
import baukit.baukit as baukit

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format = logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout
)

In [ ]:
mt = None #models.load_model

In [ ]:
relation = data.load_dataset().filter(relation_names=["country capital city"])[0].set(prompt_templates=[" {}:"])
train, test = relation.split(5)

In [ ]:
icl_prompt = functional.make_prompt(
    prompt_template = train.prompt_templates[0],
    subject= "{}",
    examples = train.samples,
    mt = mt
)
print(icl_prompt)

In [ ]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt = mt,
    test_relation=test,
    prompt_template = icl_prompt,
    batch_size = 4
)
len(test.samples)

In [ ]:
#Calculate b_r and W_r individually and average them
estimator = operators.JacobianIclMeanEstimator(
    mt = mt, h_layer=7, beta = 0.2
)

#need to call on training data to derive W_r and b_r.
operator = estimator(train)

In [ ]:
def evaluate_operator(operator, test_samples):
    pred_objects = []
    test_objects = []
    for sample in test_samples:
        test_objects.append(sample.object)
        preds = operator(sample.subject, k=3)
        pred = str(preds.predictions[0])
        #prints both the variable name and value
        print(f"{sample.subject=} -> {sample.object=} | {pred=}")
        pred_objects.append([p.token for p in preds.prediction])
    
    recall = metrics.recall(pred_objects, test_objects)
    return recall

evaluate_operator(operator, test.samples)
